In [7]:
import pandas as pd
import numpy as np
import math
import helper
import time
import datetime
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC, SVR

from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score, LeaveOneOut, ShuffleSplit, train_test_split, GridSearchCV 
from sklearn.feature_selection import SelectPercentile, SelectFromModel
import pickle    

# Подгрузка всех таблиц с данными по матчям

In [18]:
# дф с матчами и рейтингом каждой команды с патча 7.01 для Премиум
df_rating_teams_Premium = pd.read_csv('../tabel/table from Datdota/Rating teams/PREMIUM (with double Infamous) 7.00+ to 12.02.18.csv', index_col=0)
# удалить повторяющиеся матчи с Infamous, и сделать реиндекс 
df_rating_teams_Premium = df_rating_teams_Premium.drop(1).reset_index().drop('index', axis=1)

print (len(df_rating_teams_Premium))
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium[df_rating_teams_Premium['radiant_name'] == 'Digital Chaos'].index)
print (len(df_rating_teams_Premium))
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium[df_rating_teams_Premium['dire_name'] == 'Digital Chaos'].index)
print (len(df_rating_teams_Premium))
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium[df_rating_teams_Premium['radiant_team_id'] == 5065748].index)
print (len(df_rating_teams_Premium))
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium[df_rating_teams_Premium['dire_team_id'] == 5065748].index)
print (len(df_rating_teams_Premium))
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium[df_rating_teams_Premium['radiant_team_id'] == 2672298].index)
print (len(df_rating_teams_Premium))
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium[df_rating_teams_Premium['dire_team_id'] == 2672298].index)
print (len(df_rating_teams_Premium))


3729
3686
3608
3564
3541
3503
3464


In [19]:
# # дф с матчами и рейтингом каждой команды с патча 7.01 для Премиум
# df_rating_teams_Premium = pd.read_csv('../tabel/table from Datdota/Rating teams/PREMIUM (with double Infamous) 7.00+ to 12.02.18.csv', index_col=0)
# # удалить повторяющиеся матчи с Infamous, и сделать реиндекс 
# df_rating_teams_Premium = df_rating_teams_Premium.drop(1).reset_index().drop('index', axis=1)

# Убрать все рейтинг команд 
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium.loc[:, 'radiant_current_elo32':], axis=1)


# ДФ с контрпиками (head-to-head) с 6,74-7,09 и до 2018.05.06
df_contrpick = pd.read_csv(('../tabel/table from Datdota/Heah-to-head Contrpicks/'+
                'data from 6.74-7.09 with last day 2018.05.06. 7.00+ to 12.02.18.csv'), index_col=0)

# # ДФ с контрпиками (head-to-head) с 6,74-7,09 и до дня матча
# df_contrpick = pd.read_csv(('../tabel/table from Datdota/Heah-to-head Contrpicks/'+
#                 'data from 6.74-7.09 before day of match. 7.00+ to 12.02.18.csv'), index_col=0)

# ДФ с метовыми героями
df_meta_heroes = pd.read_csv('../tabel/table from Datdota/AvgElo Meta and Signatures Heroes/'+
                             'Meta 7.00+ to 12.02.18.csv', index_col=0)

# # ДФ с сигнатурками
# df_signatures_heroes = pd.read_csv('../tabel/table from Datdota/AvgElo Meta and Signatures Heroes/'+
#                                    'Sigantures 7.00+ to 12.02.18.csv', index_col=0)
# ДФ с героями как фичами
df_features_heroes = pd.read_csv('../tabel/table from Datdota/Features Heroes/'+
                                   'PREMIUM. 7.00+ to 12.02.18.csv', index_col=0)


# Создание основного ДФ для обучения и предсказания

In [20]:
# Соединение всех данных в один ДФ
main = pd.merge(df_rating_teams_Premium, df_contrpick, left_on='match_id', right_on='match_id',how='left')
main = pd.merge(main, df_meta_heroes, left_on='match_id', right_on='match_id',how='left')
# main = pd.merge(main, df_signatures_heroes, left_on='match_id', right_on='match_id',how='left')

main = pd.merge(main, df_features_heroes, left_on='match_id', right_on='match_id',how='left')

# # удаление не нужных колонок для обучения
main = main.drop(['match_id', 'start_time', 'radiant_team_id', 'radiant_score', 'dire_team_id',
           'dire_score', 'radiant_name', 'dire_name'], axis=1)
# Заполнить пустые значения на 0
main = main.fillna(0)

In [21]:
len(main)

3464

# Создание обучающих и тестовых данных

In [22]:
# удалть имена команд и сделать обучающие данные и ответы
X = main.drop(['radiant_win'], axis=1)
y = main['radiant_win']

# СДЕЛАТЬ 1 или 0 вместо true false
y = y.astype(int)

# деление на тестовые и обучающую выборку
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=0)
X_train.columns

Index(['radiant_H1elo_vs_enemies', 'radiant_H2elo_vs_enemies',
       'radiant_H3elo_vs_enemies', 'radiant_H4elo_vs_enemies',
       'radiant_H5elo_vs_enemies', 'dire_H1elo_vs_enemies',
       'dire_H2elo_vs_enemies', 'dire_H3elo_vs_enemies',
       'dire_H4elo_vs_enemies', 'dire_H5elo_vs_enemies',
       ...
       'Earth Spirit', 'Underlord', 'Terrorblade', 'Phoenix', 'Oracle',
       'Winter Wyvern', 'Arc Warden', 'Monkey King', 'Dark Willow',
       'Pangolier'],
      dtype='object', length=135)

# Решетчатый поиск на Gradient Bosting

In [47]:
%%time
# # Праметры GB
# param_grid = {'n_estimators': [50, 100, 150], 'max_depth': [10, 25, 50, 100],
#               'max_features': [10, 20, 30]}
param_grid = {'n_estimators': [100], 'max_depth': [10],
              'max_features': [20]}
grid_search_GB = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, n_jobs=-1)

grid_search_GB.fit(X_train,y_train)

print("Наилучшие значения параметров: {}".format(grid_search_GB.best_params_))
print('------------------------------------------------------------------------------------------------')
print ("Модель обученная на данных контрпиков до июня 2018")
print(classification_report(y_test, grid_search_GB.predict(X_test), target_names=['dire_win', 'radiant_win']))

Наилучшие значения параметров: {'max_depth': 10, 'max_features': 20, 'n_estimators': 100}
------------------------------------------------------------------------------------------------
Модель обученная на данных контрпиков до июня 2018
             precision    recall  f1-score   support

   dire_win       0.68      0.64      0.66       419
radiant_win       0.68      0.71      0.69       447

avg / total       0.68      0.68      0.68       866

Wall time: 10.3 s


In [56]:
# save the model to disk
filename = 'grid_search_GB_model v.2.1.a (day 12.02.18 without repeated rating teams) .sav'
pickle.dump(grid_search_GB, open(filename, 'wb'))

In [49]:
%%time
# Праметры GB
# param_grid = {'n_estimators': [25, 50, 100, 150], 'max_depth': [10, 25, 50, 100],
#               'max_features': [50, 100, 200, 300]}
param_grid = {'n_estimators': [100], 'max_depth': [10],
              'max_features': [140]}
grid_search_GB = GridSearchCV(GradientBoostingClassifier(), param_grid, cv=5, n_jobs=-1)

grid_search_GB.fit(X_train,y_train)

print("Наилучшие значения параметров: {}".format(grid_search_GB.best_params_))
print('------------------------------------------------------------------------------------------------')
print ("Модель обученная на данных контрпиков до дня матча")
print(classification_report(y_test, grid_search_GB.predict(X_test), target_names=['dire_win', 'radiant_win']))

Наилучшие значения параметров: {'max_depth': 10, 'max_features': 140, 'n_estimators': 100}
------------------------------------------------------------------------------------------------
Модель обученная на данных контрпиков до дня матча
             precision    recall  f1-score   support

   dire_win       0.58      0.58      0.58       452
radiant_win       0.60      0.60      0.60       481

avg / total       0.59      0.59      0.59       933

Wall time: 32.1 s


In [11]:
# # save the model to disk
# filename = 'grid_search_GB_model_Patch6.74+_2018.05.06.sav'
# pickle.dump(grid_search_GB, open(filename, 'wb'))

In [23]:
%%time
# ДАННЫЕ ПО HEAD-TO-HEAD ДЛЯ ВСЕХ ПАТЧЕЙ ДО ПРЕДЫДУЩЕГО ДНЯ МАТЧЯ
import xgboost as xgb
# fit model no training data
xgb_all_data = xgb.XGBClassifier()
xgb_all_data.fit(X_train, y_train)
print ("Модель обученная на данных контрпиков до июня 2018")
print(classification_report(y_test, xgb_all_data.predict(X_test), target_names=['dire_win', 'radiant_win']))

Модель обученная на данных контрпиков до июня 2018
             precision    recall  f1-score   support

   dire_win       0.70      0.67      0.68       419
radiant_win       0.70      0.73      0.71       447

avg / total       0.70      0.70      0.70       866

Wall time: 315 ms


In [70]:
# save the model to disk
filename = 'xgb_all_data v.2.1.a (day 12.02.18 without repeated rating teams) .sav'
pickle.dump(xgb_all_data, open(filename, 'wb'))

In [48]:
%%time
# ДАННЫЕ ПО HEAD-TO-HEAD ДЛЯ ВСЕХ ПАТЧЕЙ ДО ПРЕДЫДУЩЕГО ДНЯ МАТЧЯ
import xgboost as xgb
# fit model no training data
xgb_all_data = xgb.XGBClassifier()
xgb_all_data.fit(X_train, y_train)
print ("Модель обученная на данных контрпиков до дня матча")
print(classification_report(y_test, xgb_all_data.predict(X_test), target_names=['dire_win', 'radiant_win']))

Модель обученная на данных контрпиков до дня матча
             precision    recall  f1-score   support

   dire_win       0.61      0.59      0.60       452
radiant_win       0.62      0.64      0.63       481

avg / total       0.62      0.62      0.62       933

Wall time: 307 ms


# Проверка на новых данных

In [37]:
# дф с матчами и рейтингом каждой команды с патча 7.01 для Премиум
df_rating_teams_Premium = pd.read_csv('../tabel/table from Datdota/Rating teams/'+
                                      'PREMIUM on 13.02.18 to 12.03.18.csv', index_col=0)
# Убрать все рейтинг команд 
df_rating_teams_Premium = df_rating_teams_Premium.drop(
    df_rating_teams_Premium.loc[:, 'radiant_H1':], axis=1)

# ДФ с контрпиками (head-to-head) с 6,74-7,09 и до 2018.05.06
df_contrpick = pd.read_csv(('../tabel/table from Datdota/Heah-to-head Contrpicks/'+
                'data from 6.74-7.10 with last day 2018.05.06. on 13.02.18 to 12.03.18.csv'), index_col=0)

# # ДФ с контрпиками (head-to-head) с 6,74-7,09 и до дня матча
# df_contrpick = pd.read_csv(('../tabel/table from Datdota/Heah-to-head Contrpicks/'+
#                 'data from 6.74-7.10 before day of match. on 13.02.18 to 12.03.18.csv'), index_col=0)

# ДФ с метовыми героями
df_meta_heroes = pd.read_csv('../tabel/table from Datdota/AvgElo Meta and Signatures Heroes/'+
                             'Meta on 13.02.18 to 12.03.18.csv', index_col=0)

# # ДФ с сигнатурками
# df_signatures_heroes = pd.read_csv('../tabel/table from Datdota/AvgElo Meta and Signatures Heroes/'+
#                                    'Signatures on 13.02.18 to 12.03.18.csv', index_col=0)
# ДФ с героями как фичами
df_features_heroes = pd.read_csv('../tabel/table from Datdota/Features Heroes/'+
                                   'PREMIUM. on 13.02.18 to 12.03.18.csv', index_col=0)


In [39]:
# Соединение всех данных в один ДФ
test = pd.merge(df_rating_teams_Premium, df_contrpick, left_on='match_id', right_on='match_id',how='left')
test = pd.merge(test, df_meta_heroes, left_on='match_id', right_on='match_id',how='left')
# test = pd.merge(test, df_signatures_heroes, left_on='match_id', right_on='match_id',how='left')
test = pd.merge(test, df_features_heroes, left_on='match_id', right_on='match_id',how='left')


# # удаление не нужных колонок для обучения
test = test.drop(['match_id', 'start_time', 'radiant_team_id', 'radiant_score', 'dire_team_id',
           'dire_score' ], axis=1)
test = test.dropna()

In [41]:
# Создание контрольной выборки
# удалть имена команд и сделать обучающие данные и ответы
X_contr = test.drop(['radiant_win'], axis=1)
y_contr = test['radiant_win']

# СДЕЛАТЬ 1 или 0 вместо true false
y_contr = y_contr.astype(int)

In [52]:
print ("Модель обученная на данных контрпиков до июня 2018")
print(classification_report(y_contr, grid_search_GB.predict(X_contr), target_names=['dire_win', 'radiant_win']))

Модель обученная на данных контрпиков до июня 2018
             precision    recall  f1-score   support

   dire_win       0.54      0.60      0.57       164
radiant_win       0.67      0.62      0.64       216

avg / total       0.61      0.61      0.61       380



In [42]:
print ("Модель обученная на данных контрпиков до июня 2018")
print(classification_report(y_contr, xgb_all_data.predict(X_contr), target_names=['dire_win', 'radiant_win']))

Модель обученная на данных контрпиков до июня 2018
             precision    recall  f1-score   support

   dire_win       0.57      0.59      0.58       164
radiant_win       0.68      0.67      0.67       216

avg / total       0.64      0.63      0.63       380



In [38]:
print ("Модель обученная на данных контрпиков до дня матча")
print(classification_report(y_contr, grid_search_GB.predict(X_contr), target_names=['dire_win', 'radiant_win']))

Модель обученная на данных контрпиков до дня матча
             precision    recall  f1-score   support

   dire_win       0.50      0.48      0.49       164
radiant_win       0.61      0.63      0.62       216

avg / total       0.56      0.57      0.56       380



In [39]:
print ("Модель обученная на данных контрпиков до дня матча")
print(classification_report(y_contr, xgb_all_data.predict(X_contr), target_names=['dire_win', 'radiant_win']))

Модель обученная на данных контрпиков до дня матча
             precision    recall  f1-score   support

   dire_win       0.50      0.51      0.51       164
radiant_win       0.62      0.62      0.62       216

avg / total       0.57      0.57      0.57       380



In [40]:
test

,radiant_win,radiant_H1elo_vs_enemies,radiant_H2elo_vs_enemies,radiant_H3elo_vs_enemies,radiant_H4elo_vs_enemies,radiant_H5elo_vs_enemies,dire_H1elo_vs_enemies,dire_H2elo_vs_enemies,dire_H3elo_vs_enemies,dire_H4elo_vs_enemies,...,radiant_H1_AvgElo_y,radiant_H2_AvgElo_y,radiant_H3_AvgElo_y,radiant_H4_AvgElo_y,radiant_H5_AvgElo_y,dire_H1_AvgElo_y,dire_H2_AvgElo_y,dire_H3_AvgElo_y,dire_H4_AvgElo_y,dire_H5_AvgElo_y
0,True,-0.896954,20.450689,18.553208,-5.023051,8.672485,-20.476676,4.936129,0.585725,-17.752355,...,2.779152,6.640791,2.352480,-3.606404,2.086108,-19.817246,-5.294288,0.895989,-3.009805,0.055967
1,True,15.788671,-6.333362,1.497776,-3.393039,-2.657813,-9.056077,19.281367,-14.190674,14.594612,...,0.280441,2.086108,0.953798,-2.615389,3.058034,-2.176290,-1.877006,-1.026533,-1.255653,-0.805565
2,True,-7.170763,-2.488786,20.867928,9.208293,6.121526,-9.822999,-12.690407,-10.077451,0.792141,...,-2.176290,-3.533811,6.500131,-1.050380,-0.039413,-2.615389,2.710005,1.847699,3.058034,-5.294288
3,True,14.267182,-5.980386,24.646601,17.176353,11.790325,-9.718243,-12.714589,-12.259485,-8.221015,...,-0.092653,-0.823236,-4.599451,1.967669,0.891692,-1.843531,-2.615389,-2.176290,-3.606404,-3.576825
4,True,7.432770,4.331540,-5.047987,-2.117185,-7.823640,-5.256943,-5.788481,8.334696,13.517247,...,6.846729,2.367828,2.512874,-0.595824,1.847699,1.967669,-5.294288,0.953798,-0.092653,-0.098350
5,False,-13.535479,3.506864,22.959921,0.522086,15.558027,1.242087,-7.870235,-6.495684,-7.990047,...,-2.176290,2.512874,2.509039,0.103990,-0.092653,-0.595824,-5.294288,1.472287,-9.088761,-3.009805
6,False,-12.797682,0.919897,4.127433,-13.892132,-2.548005,17.042227,4.618905,0.480001,4.401435,...,-2.176290,6.218296,2.512874,-5.293334,-2.615389,-0.039413,2.352480,1.847699,2.779152,-7.866778
7,True,1.398895,5.931781,-11.814809,1.300593,-9.247498,14.135407,-3.451320,-1.146752,1.253845,...,1.967669,-5.294288,-7.866778,0.891692,-3.009805,3.058034,0.470684,-3.606404,2.830216,2.512874
8,True,-0.497542,13.718512,22.177047,5.599078,12.662686,-1.740842,-15.042457,-7.504577,-14.328693,...,1.847699,-1.050380,0.953798,3.058034,-5.293334,0.103990,-2.615389,-0.098350,-0.153075,-0.823236
9,True,-11.156332,6.214052,-3.343665,12.575515,-16.296612,8.040221,12.540963,-7.650231,6.762579,...,-0.894266,6.374394,0.953798,2.352480,-3.066346,1.967669,-1.877006,1.472287,2.710005,0.895989
